In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
Train=pd.read_csv('/kaggle/input/new-training-data-from-1990-2015/new_train.csv')
val=pd.read_csv('/kaggle/input/new-val/validation (1).csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [2]:
unique = Train['Country Name'].unique()

In [3]:
#CO2 emissions (metric tons per capita)
Train=Train.replace('CO2 emissions (metric tons per capita)',' ZCO2 emissions (metric tons per capita)') 
val=val.replace('CO2 emissions (metric tons per capita)',' ZCO2 emissions (metric tons per capita)')

In [4]:
j= pd.read_csv('/kaggle/input/missing-prediction/data.csv')

In [5]:
missing_value2 = j.values

In [6]:
Train = Train.fillna(.0102)
Test=Test.replace('..', .01020) 

In [7]:
val = val.fillna(.0102)

In [8]:
val=val.replace('..', .01020)

In [9]:
import pandas as pd
import warnings
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

2024-03-24 20:45:05.482439: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 20:45:05.482570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 20:45:05.664794: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
checkpoint_filepath = '/kaggle/working/best_model.keras'

        # Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,
                             monitor='val_loss',  # Monitor validation loss
                             mode='min',         # Mode: minimize the monitored quantity
                             save_best_only=True,  # Save only the best model
                             # Save only the model weights
                             verbose=1)

In [11]:
#trainig and inferencing with LogisticRegression and ARIMA
import pandas as pd
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

k=0
for i in unique:
    transposed_df=0
    fg = Train[Train['Country Name'] == 'Afghanistan'].reset_index(drop=True)
    fg=fg.sort_values(by='Indicator Name', ascending=False).reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator Name']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
#print(Xtrain)
    Ytrain=transposed_df[11].astype(float)
    fgt = Test[Test['Country Name'] == 'Afghanistan'].reset_index(drop=True)
    fgt=fgt.sort_values(by='Indicator', ascending=False).reset_index(drop=True)
    columns_to_drop = [ 'Country Name','Indicator']
    fgt = fgt.drop(columns_to_drop, axis=1)
    transposed_df = fgt.transpose()
        #print(transposed_df)
    Xtest = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
    fgk = val[val['Country Name'] == 'Afghanistan'].reset_index(drop=True)
    fgk=fgk.sort_values(by='Indicator', ascending=False).reset_index(drop=True)
    columns_to_drop = ['Country Name','Indicator']
    fgk = fgk.drop(columns_to_drop, axis=1)
    transposed_df = fgk.transpose()
    #print(fg)
    #print('done for now')
    Xval = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
#print(Xtrain)
    Yval=transposed_df[11].astype(float)
    x_v=Xval.values
    y_v=Yval.values



#print(Xtest)
    Xtrain
    
    y_train=Ytrain.values
    X_train=Xtrain.values
    xt=Xtrain.values
    model = Sequential([
              LSTM(64, return_sequences=True, input_shape=(Xtrain.shape[1], 1)),
              Dropout(0.2),  # Adding dropout for regularization
              LSTM(64, return_sequences=True),
              Dropout(0.2),
              LSTM(64),  # Last LSTM layer without return sequences
              Dropout(0.2),
              Dense(32, activation='relu'),  # Adding a dense layer for additional processing
              Dense(1)  # Output layer
              ])
    model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
    Xtrain = xt.reshape((xt.shape[0], xt.shape[1], 1))
    x_v1=x_v.reshape((x_v.shape[0], x_v.shape[1], 1))
    model.fit(  X_train,  y_train, epochs=22, batch_size=1,validation_data=(x_v1.astype(float), y_v),callbacks=[checkpoint])
    forecast = model.predict(Xtest.astype(float))
    model2 = ARIMA(Ytrain.astype(float), order=(0,1,0))  # Adjust order as needed
    model_fit2 = model2.fit()
    print(forecast)
    
    forecast2 = model_fit2.forecast(steps=16)
       
    row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


    new_values = [i, forecast[0][0], forecast[1][0],forecast[2][0], forecast[3][0], forecast[4][0],forecast2[15]]
    sample_submission.loc[row_index] = new_values
        

Epoch 1/22
25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0261
Epoch 1: val_loss improved from inf to 0.01810, saving model to /kaggle/working/best_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 0.0256 - val_loss: 0.0181
Epoch 2/22
24/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0126
Epoch 2: val_loss improved from 0.01810 to 0.01157, saving model to /kaggle/working/best_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0128 - val_loss: 0.0116
Epoch 3/22
25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0090
Epoch 3: val_loss improved from 0.01157 to 0.00280, saving model to /kaggle/working/best_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0092 - val_loss: 0.0028
Epoch 4/22
21/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0197
Epoch 4: val_loss did not improve from 0.00280
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0185 - val_loss: 0.0051
Epoch 5/22
25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0113
Epoch 5: val_loss did not improve

In [12]:
#trainig and inferencing with LogisticRegression and ARIMA
import pandas as pd
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


for i in missing_value2:
    
    
    fgt = Test[Test['Country Name'] == i[0]].reset_index(drop=True)
    columns_to_drop = [ 'Country Name','Indicator']
    fgt = fgt.drop(columns_to_drop, axis=1)
    transposed_df = fgt.transpose()
    Xtest = transposed_df[[0,1, 2,3,4,9,10]]
    transposed_df=0
    fg = Train[Train['Country Name'] == i[0]].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator Name']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,9,10]].astype(float)
    xt=Xtrain.values
    Ytrain=transposed_df[11].astype(float)
    
    
    loaded_model = tf.keras.models.load_model("/kaggle/input/deep-v4-22/my_lstm_model (4).h5")
   
    forecast = loaded_model.predict(Xtest.astype(float))
    df = pd.DataFrame(forecast)
    model2 = ARIMA(df[0], order=(0,1,0))  # Adjust order as needed
    model_fit = model2.fit()

    
    forecast2 = model_fit.forecast(steps=16)
        
    row_index = sample_submission[sample_submission.eq(i[0]).any(axis=1)].index[0]


    new_values = [i[0], forecast[0][0], forecast[1][0],forecast[2][0], forecast[3][0], forecast[4][0],forecast2[15]]
    sample_submission.loc[row_index] = new_values
        

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [13]:
sample_submission.to_csv('submission.csv', index=False)